In [1]:
import sys
parentdir = 'C:\\Users\\anaxa\\source\\BAMT'
sys.path.insert(0,parentdir) 

In [2]:
import ipywidgets as widgets
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from scipy.spatial import distance
from preprocess.discretization import get_nodes_type, get_nodes_sign
import networkx as nx
import matplotlib.pyplot as plt
from bayesian.train_bn import structure_learning, parameter_learning
from bayesian.save_bn import save_structure, save_params, read_structure, read_params
from bayesian.sampling import generate_synthetics
from external.libpgm.hybayesiannetwork import HyBayesianNetwork
import ast


Bad key "text.kerning_factor" on line 4 in
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
def get_n_nearest_by_corr(data, columns, number_res=5):
    groups = []
    for c in columns:
        close_ind = data[c + ' act'].sort_values(ascending=False).index
        close_res = []
        for element in close_ind:
            if element.split()[-1] == 'res':
                close_res.append(element)
        groups.append([c]+close_res[0:number_res])
    return groups


def get_n_nearest_by_dist(data, columns, number_res=5):
    groups = []
    for c in columns:
        close_ind = data[c+' act'].sort_values().index
        close_res = []
        for element in close_ind:
            if element.split()[-1] == 'res':
                close_res.append(element)
        groups.append([c]+close_res[0:number_res])
    return groups

In [4]:
df = pd.read_csv('file3_fact.csv')

df_empty_column = []
for c in df.columns:
    if (df[c] == 0).all():
        df_empty_column.append(c)


columns = [c for c in df.columns.tolist() if c not in df_empty_column]
df = df[columns]

df = df.astype(float)

In [5]:
act = [c for c in df.columns if c.split()[-1]=='act']
res = [c for c in df.columns if c.split()[-1]=='res']

In [6]:
corr = df.corr('spearman')

In [7]:
df_normed = pd.DataFrame(data = MinMaxScaler().fit_transform(df.values), columns=df.columns)

In [8]:
distance_df = pd.DataFrame(data=np.zeros((len(res), len(act))), columns=act)
distance_df.index = res

In [9]:
for c1 in act:
    non_emty_act_ind = []
    for i in df.index:
        if df.loc[i,c1] != 0:
            non_emty_act_ind.append(i)
    for c2 in res:
        distance_df.loc[c2, c1] = distance.euclidean(df_normed.loc[non_emty_act_ind, c1].values, df_normed.loc[non_emty_act_ind, c2].values)


## Предрасчёт всех сетей

In [10]:
# for c1 in act:
#     print(c1)
#     for n in range (10):
#         print(n)
#         close_n = n + 1
#         group1 = get_n_nearest_by_corr(corr, [c1], close_n)[0]
#         group2 = get_n_nearest_by_dist(distance_df, [c1], close_n)[0]
#         start_dag = []
#         for i in range(1, len(group1)):
#             start_dag.append((group1[0], group1[i]))
       
#         white_list = []
#         for r1 in group1[1:len(group1)]:
#             for r2 in group1[1:len(group1)]:
#                 if r1 != r2:
#                     white_list.append((r1, r2))
#                     white_list.append((r2, r1))
#         bn1 = structure_learning(df[group1],'HC', get_nodes_type(df[group1]), 'K2',  
#             white_list=white_list, init_edges=start_dag, remove_init_edges=False)
#         save_structure(bn1, 'corr_'+c1+'_'+str(close_n))
#         params1 = parameter_learning(df[group1], get_nodes_type(df[group1]), bn1, 'mix')
#         save_params(params1, 'corr_'+c1+'_'+str(close_n))


#         start_dag = []
#         for i in range(1, len(group2)):
#             start_dag.append((group2[0], group2[i]))
       
#         white_list = []
#         for r1 in group2[1:len(group2)]:
#             for r2 in group2[1:len(group2)]:
#                 if r1 != r2:
#                     white_list.append((r1, r2))
#                     white_list.append((r2, r1))
#         bn2 = structure_learning(df[group2],'HC', get_nodes_type(df[group2]), 'K2',  
#             white_list=white_list, init_edges=start_dag, remove_init_edges=False)
#         save_structure(bn2, 'dist_'+c1+'_'+str(close_n))
#         params2 = parameter_learning(df[group2], get_nodes_type(df[group2]), bn2, 'mix')
#         save_params(params2, 'dist_'+c1+'_'+str(close_n))
#     print('------------')
        

        

In [54]:
work = 'Бурение скважин'
n_close = 5
skel = read_structure('corr_'+work+' act' +'_'+str(n_close))
params = read_params('corr_'+work+' act' +'_'+str(n_close))
bn = HyBayesianNetwork(skel, params)

In [55]:
sample = generate_synthetics(bn, get_nodes_sign(df[bn.V]), 'mix', 50)

In [56]:
sample

,Бурение скважин act,Механизаторы res,Автобетоносмеситель res,"Электрик, электромонтажник res",Тягач с полуприцепом res,Маляр res
0,60.000000,48.000000,2.000000,2.000000,4.000000,12.000000
1,141.724390,39.158046,1.631585,1.631585,0.221073,0.663219
2,60.000000,48.000000,2.000000,2.000000,4.000000,12.000000
3,60.000000,48.000000,2.000000,2.000000,4.000000,12.000000
4,180.844722,54.844356,2.285181,2.285181,2.312581,6.937742
5,188.766597,56.678115,2.361588,2.361588,2.557082,7.671246
6,171.008992,51.862250,2.160927,2.160927,1.914967,5.744900
7,60.000000,48.000000,2.000000,2.000000,4.000000,12.000000
8,60.000000,48.000000,2.000000,2.000000,4.000000,12.000000
9,60.000000,48.000000,2.000000,2.000000,4.000000,12.000000


In [57]:

w1 = widgets.Dropdown(
    options=['Бурение скважин', 'Изготовление металлоконструкицй', 'Изготовление свай', 
    'Монтаж металлоконструкций', 'Монтаж оголовников свай', 'Монтаж опор', 'Надземная прокладка трубопровода',
    'Сварка трубопровода', 'Срезка свай', 'Устройство свайного основания'],
    description='Работа:',value='Бурение скважин',
    disabled=False)
w2 = widgets.IntSlider(
    value=7,
    min=1,
    max=10,
    step=1,
    description='Кол-во ближайших ресурсов:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
text1 = widgets.Textarea()
text2 = widgets.Textarea()
def value_changed1(change):
    text1.value = str(get_n_nearest_by_corr(corr, [change.new], w2.value)[0][1:w2.value+1])
    text2.value = str(get_n_nearest_by_dist(distance_df, [change.new], w2.value)[0][1:w2.value+1])


def value_changed2(change):
    text1.value = str(get_n_nearest_by_corr(corr, [w1.value], change.new)[0][1:change.new+1])
    text2.value = str(get_n_nearest_by_dist(distance_df, [w1.value], change.new)[0][1:change.new+1])
w1.observe(value_changed1, 'value')
w2.observe(value_changed2, 'value')

w3 = widgets.RadioButtons(
    options=['Correlation', 'Euclidean distance'],
#    layout={'width': 'max-content'}, # If the items' names are long
    disabled=False
)
label1 = widgets.Label('Корреляция Спирмена:')
label2 = widgets.Label('Евклидово расстояние:')

img = widgets.Image()

def value_changed3(change):
    bn = dict()

    if change.new == 'Correlation':
        work = w1.value
        n_close = w2.value
        skel = read_structure('corr_'+work+' act' +'_'+str(n_close))
        params = read_params('corr_'+work+' act' +'_'+str(n_close))
        bn = HyBayesianNetwork(skel, params)

        
    else:
        work = w1.value
        n_close = w2.value
        skel = read_structure('dist_'+work+' act' +'_'+str(n_close))
        params = read_params('dist_'+work+' act' +'_'+str(n_close))
        bn = HyBayesianNetwork(skel, params)

    



    G = nx.DiGraph()
    G.add_nodes_from(bn.V)
    G.add_edges_from(bn.E)
    

    nx.draw(G, with_labels=True, pos=nx.circular_layout(G))

  
    plt.savefig('plotgraph.png', dpi=300)

 

    file = open("plotgraph.png", "rb")
    image = file.read()
    img.value = image





w3.observe(value_changed3, 'value')

def on_button_clicked(b):
    evidence = dict()
    if w4.value == '':
        evidence = None
    else:
        for element in w4.value.split(','):
            evidence[element.split(':')[0]] = int(element.split(':')[1])
 
    if w3.value == 'Correlation':
        work = w1.value
        n_close = w2.value
        skel = read_structure('corr_'+work+' act' +'_'+str(n_close))
        params = read_params('corr_'+work+' act' +'_'+str(n_close))
        bn = HyBayesianNetwork(skel, params)
    else:
        work = w1.value
        n_close = w2.value
        skel = read_structure('dist_'+work+' act' +'_'+str(n_close))
        params = read_params('dist_'+work+' act' +'_'+str(n_close))
        bn = HyBayesianNetwork(skel, params)
    sample = generate_synthetics(bn, get_nodes_sign(df[bn.V]), 'mix', 50, evidence=evidence)
    res_estimation = dict()
    for node in bn.V:
        if node.split()[-1] == 'res':
            res_estimation[node] = sample[node].mean()
    w5.value = str(res_estimation)
  

   


  

w4 = widgets.Text(placeholder='Задать количество работ и ресурсов')
button = widgets.Button(description="Оценить ресурсы")
w5 = widgets.Textarea(placeholder='Результат оценки ресурсов')
button.on_click(on_button_clicked)
widgets.VBox([w1, w2, label1, text1, label2, text2, w3, img, w4, button, w5])
